In [1]:
import sys
sys.path.append(r"C:\Users\kandu\VITON-HD")

In [2]:
import os
import torch
import torch.optim as optim
from datasets import VITONDataset, VITONDataLoader
from networks import SegGenerator, GMM, ALIASGenerator
# Paths
data_path = "C:/Joseph"
checkpoint_dir = "C:/Users/kandu/VITON-HD/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

opt = type('opt', (object,), {})()  
opt.dataset_dir = data_path
opt.dataset_list = "C:/Joseph/train_pairs.txt"
opt.load_height = 1024
opt.load_width = 768
opt.semantic_nc = 13
opt.ngf = 64
opt.num_upsampling_layers = 'most'
opt.init_type = 'normal'
opt.init_variance = 0.02
opt.dataset_mode = "train"
opt.shuffle = True  
opt.batch_size = 1  
opt.workers = 1  
opt.grid_size = 8 
opt.norm_G = 'aliasmask'

In [3]:
dataset = VITONDataset(opt)
data_loader = VITONDataLoader(opt, dataset)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seg = SegGenerator(opt, input_nc=opt.semantic_nc + 8, output_nc=opt.semantic_nc).to(device)
gmm = GMM(opt, inputA_nc=7, inputB_nc=3).to(device)
alias = ALIASGenerator(opt, input_nc=9).to(device)

Network [SegGenerator] was created. Total number of parameters: 34.5 million. To see the architecture, do print(network).
Network [ALIASGenerator] was created. Total number of parameters: 100.7 million. To see the architecture, do print(network).


In [4]:
learning_rate = 0.0002  
optimizer_G = optim.Adam(seg.parameters(), lr=learning_rate)
optimizer_GMM = optim.Adam(gmm.parameters(), lr=learning_rate)
optimizer_alias = optim.Adam(alias.parameters(), lr=learning_rate)

In [5]:
num_epochs = 10 
for epoch in range(num_epochs):
    for i, inputs in enumerate(data_loader.data_loader):
        img_agnostic = inputs['img_agnostic'].to(device)

        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(data_loader.data_loader)}]')

    # Save checkpoints at the end of each epoch
    torch.save(seg.state_dict(), os.path.join(checkpoint_dir, 'seg_trail.pth'))
    torch.save(gmm.state_dict(), os.path.join(checkpoint_dir, 'gmm_trail.pth'))
    torch.save(alias.state_dict(), os.path.join(checkpoint_dir, 'alias_trail.pth'))

print("Training complete!")

Epoch [1/10], Step [10/25]
Epoch [1/10], Step [20/25]
Epoch [2/10], Step [10/25]
Epoch [2/10], Step [20/25]
Epoch [3/10], Step [10/25]
Epoch [3/10], Step [20/25]
Epoch [4/10], Step [10/25]
Epoch [4/10], Step [20/25]
Epoch [5/10], Step [10/25]
Epoch [5/10], Step [20/25]
Epoch [6/10], Step [10/25]
Epoch [6/10], Step [20/25]
Epoch [7/10], Step [10/25]
Epoch [7/10], Step [20/25]
Epoch [8/10], Step [10/25]
Epoch [8/10], Step [20/25]
Epoch [9/10], Step [10/25]
Epoch [9/10], Step [20/25]
Epoch [10/10], Step [10/25]
Epoch [10/10], Step [20/25]
Training complete!
